In [ ]:
!pip install ISLP

In [ ]:
#We start by importing our libraries at this top level; these are all imports we have seen in previous labs.

import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)

In [ ]:
#Now we are ready to load the Smarket data.

Smarket = load_data('Smarket')
Smarket

In [ ]:
#We compute the correlation matrix using the corr() method for data frames, which produces a matrix that contains all of the pairwise correlations among the variables.
Smarket.corr(numeric_only=True)


In [ ]:
Smarket.plot(y='Volume');

In [ ]:
#We begin by performing LDA on the Smarket data, using the function LinearDiscriminantAnalysis(), which we have abbreviated LDA()
allvars = Smarket.columns.drop(['Today', 'Direction', 'Year'])
design = MS(allvars)
X = design.fit_transform(Smarket)
y = Smarket.Direction == 'Up'


train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape


In [ ]:
model = MS(['Lag1', 'Lag2']).fit(Smarket)
X = model.transform(Smarket)
X_train, X_test = X.loc[train], X.loc[~train]

D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]

X_train, X_test = [M.drop(columns=['intercept'])
                   for M in [X_train, X_test]]

lda = LDA(store_covariance=True)

lda.fit(X_train, L_train)

In [ ]:
#These values provide the linear combination of Lag1 and Lag2 that are used to form the LDA decision rule.
lda.means_
lda.classes_
#The estimated prior probabilities are stored in the priors_ attribute. 
lda.priors_

In [ ]:
lda_pred = lda.predict(X_test)


In [ ]:
confusion_table(lda_pred, L_test)
